# Springboard Data Science Capstone 2
Identifying Credit Card Fraud: 1. Data Wrangling

Mark Cohen

2023-3-7

## 1 Setup

The structure of the project directory is as follows:
- data
    - raw
    - processed
- src
- notebooks
- models
- reports

The `src` directory includes python scripts that define utility functions needed at various steps of the project. This data wrangling notebook will use the `data_utils.py`, which provides functions to download and load the project's data.

**NOTE:** because of its size, the raw data is not stored in the github repository. Instead, the zipped data is mirrored on my Google Drive account. See the file `data source and license.txt` for the original source and license. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
sys.path.append("../src")
import data_utils

In [3]:
# Confirming that data is present locally, or downloading and unzipping if not.
data_utils.raw_data_on_disk()

Downloaded ../tmp/data_archive.zip
Unzipping data files.


## 2 Inspecting the data files

### 2.1 A sample transaction record

The data set includes the transaction records for a single customer separated out into its own csv file. It will be useful to look at this file to get a sense of the format of the data.

In [4]:
df_user0 = data_utils.read_sample_transactions()

As seen below, the dataset includes almost 20,000 transactions for this one user, and each transaction record consists of 15 features.

The column names are inconvenient for data frame indexing. They should be reformatted in snake case with no special characters.

We can also see already that some of the data types will need to be adjusted:
1. Amount is a string, including currency marks.
1. Zip is a float rather than int or string.
1. The target feature is a text `yes`/`no` rather than an integer or boolean.

`MCC` is "Merchant Category Code." For example: `5411` represents "Grocery Stores and Supermarkets." See the following document from visa: https://usa.visa.com/content/dam/VCOM/download/merchants/visa-merchant-data-standards-manual.pdf.

Finally, there are a substantial number of missing values in the `Merchant State` and `Zip` columns. In the `Errors?` column most rows have no value, which probably means there is no error.

In [5]:
print(df_user0.info())
print(df_user0.head(3).T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19963 entries, 0 to 19962
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   User            19963 non-null  int64  
 1   Card            19963 non-null  int64  
 2   Year            19963 non-null  int64  
 3   Month           19963 non-null  int64  
 4   Day             19963 non-null  int64  
 5   Time            19963 non-null  object 
 6   Amount          19963 non-null  object 
 7   Use Chip        19963 non-null  object 
 8   Merchant Name   19963 non-null  int64  
 9   Merchant City   19963 non-null  object 
 10  Merchant State  18646 non-null  object 
 11  Zip             18316 non-null  float64
 12  MCC             19963 non-null  int64  
 13  Errors?         574 non-null    object 
 14  Is Fraud?       19963 non-null  object 
dtypes: float64(1), int64(7), object(7)
memory usage: 2.3+ MB
None
                                  0                    1   

In [6]:
# Updating the column names
old_colnames = df_user0.columns
print(f"Old column names: {old_colnames}")
new_colnames = (
    old_colnames
    .str.lower()
    .str.replace(' ', '_', regex=False)
    .str.replace('?', '', regex=False)
)
print(f"New column names: {new_colnames}")
df_user0.columns = new_colnames

Old column names: Index(['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?'],
      dtype='object')
New column names: Index(['user', 'card', 'year', 'month', 'day', 'time', 'amount', 'use_chip',
       'merchant_name', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors', 'is_fraud'],
      dtype='object')


Looking next at descriptive statistics for the numeric features, a few points stand out.
1. The data for this user includes transactions from 5 distinct cards.
1. The data covers a long period of time: from 2002 to 2020. One possibility might be to subset the data on time rather than by customers, e.g. restrict it to a period of a few years.
1. In addition to missing values, the minimum value of the zip code column is not a valid zip code.

In [7]:
df_user0.describe()

,user,card,year,month,day,merchant_name,zip,mcc
count,19963.0,19963.000000,19963.000000,19963.000000,19963.000000,1.996300e+04,18316.000000,19963.000000
mean,0.0,1.910735,2011.011922,6.568101,15.743876,7.825653e+17,88812.744922,5617.940239
std,0.0,1.237763,5.048146,3.477497,8.801378,4.040602e+18,13711.491085,707.982901
min,0.0,0.000000,2002.000000,1.000000,1.000000,-9.179793e+18,1012.000000,1711.000000
25%,0.0,0.000000,2007.000000,4.000000,8.000000,-1.288082e+18,91750.000000,5311.000000
50%,0.0,2.000000,2011.000000,7.000000,16.000000,8.384250e+17,91750.000000,5499.000000
75%,0.0,3.000000,2015.000000,10.000000,23.000000,4.060647e+18,91752.000000,5912.000000
max,0.0,4.000000,2020.000000,12.000000,31.000000,9.137769e+18,99504.000000,9402.000000


Looking first at the `zip` column, most of the rows with missing data represent online transactions. Transactions outside of the United States are recorded such that there is no zip code and the country name is stored in the `merchant_state` column.

In [8]:
missing_zip = df_user0.loc[df_user0.zip.isna(),["merchant_city", "merchant_state", "zip"]]
print(missing_zip.merchant_city.value_counts())
print()
print(missing_zip.loc[missing_zip.merchant_state != 'ONLINE', 'merchant_state'].value_counts())

ONLINE            1317
Cancun             112
Manila              46
Kingston            46
Cabo San Lucas      34
Rome                32
Tallinn             13
Tokyo               12
Beijing             11
Shanghai             7
Lisbon               6
Zurich               5
Santo Domingo        4
Toronto              2
Name: merchant_city, dtype: int64

Mexico                146
Philippines            46
Jamaica                46
Italy                  32
China                  18
Estonia                13
Japan                  12
Portugal                6
Switzerland             5
Dominican Republic      4
Canada                  2
Name: merchant_state, dtype: int64


As for the zip codes that seem to be too short, this is apparently because leading zeros (i.e. zip codes from northeastern states) have been dropped. This does not impact the validity or usability of the data, so it will be left as is for now.

In [9]:
low_zip = df_user0[df_user0.zip < 10000]
print(low_zip.merchant_state.unique())

['NJ' 'CT' 'MA']


Moving on to the textual columns, it turns out that the `amount` column just appends `$` to every value. Accordingly, this can be removed and the values converted to float.

In [10]:
print(f"The first character of the value column:\n {df_user0.amount.str.get(0).value_counts()}")

df_user0.amount = df_user0.amount.str.slice(1).astype('float')
df_user0.amount.describe()

The first character of the value column:
 $    19963
Name: amount, dtype: int64


count    19963.000000
mean        81.299989
std         94.159093
min       -499.000000
25%         36.630000
50%         69.450000
75%        125.680000
max       1409.400000
Name: amount, dtype: float64

This reveals an additional issue: some of the transaction values are negative. These appear to be refunds: note how the 2nd example immediately follows a transaction for the same (but positive) amount from the same merchant.

In [11]:
neg_val = df_user0[df_user0.amount <= 0]
print(neg_val.head(3).T)
print(df_user0.iloc[71:73,:].T)

                                32                    72   \
user                              0                     0   
card                              0                     0   
year                           2002                  2002   
month                             9                     9   
day                              11                    25   
time                          13:17                 13:14   
amount                        -99.0                -100.0   
use_chip          Swipe Transaction     Swipe Transaction   
merchant_name   2027553650310142703  -1288082279022882052   
merchant_city             Mira Loma              La Verne   
merchant_state                   CA                    CA   
zip                         91752.0               91750.0   
mcc                            5541                  5499   
errors                          NaN                   NaN   
is_fraud                         No                    No   

                       

The separate columns for year, month, day, and time can be combined into a single Pandas Timestamp column.

In [12]:
def row_to_timestamp(row: pd.Series):
    dt_str = f"{row.year}-{row.month:02}-{row.day:02}T{row.time}"
    return pd.Timestamp(dt_str)

df_user0['timestamp'] = df_user0.apply(row_to_timestamp, axis=1)
# Confirm the years match
print("Mismatched years:", (df_user0.year != df_user0.timestamp.dt.year).sum())

Mismatched years: 0


Next, let's look at `use_chip`. It turns out there are only three values, corresponding to swipe, chip, and online. We can clean this up by renaming the column more intuitively, and converting into categories without redundant names.

In [13]:
print(df_user0.use_chip.value_counts(), df_user0.use_chip.isna().sum())

Swipe Transaction     15840
Chip Transaction       2808
Online Transaction     1315
Name: use_chip, dtype: int64 0


In [14]:
tx_type = df_user0.use_chip.str.strip(" Transaction").str.lower().astype("category")
print(tx_type.cat.categories)
df_user0['tx_type'] = tx_type

Index(['chip', 'online', 'swipe'], dtype='object')


In [15]:
df_user0.drop(columns = ["use_chip"], inplace=True)

Finally, the target feature indicating fraud is a binary 'Yes' or 'No'. It will be easier to work with as a boolean.

In [16]:
print(df_user0.is_fraud.value_counts())

df_user0.is_fraud = df_user0.is_fraud == 'Yes'

print(df_user0.is_fraud.value_counts())

No     19936
Yes       27
Name: is_fraud, dtype: int64
False    19936
True        27
Name: is_fraud, dtype: int64


This is what the head of the data frame looks like after processing:

In [17]:
df_user0.head(2).T

,0,1
user,0,0
card,0,0
year,2002,2002
month,9,9
day,1,1
time,06:21,06:42
amount,134.09,38.48
merchant_name,3527213246127876953,-727612092139916043
merchant_city,La Verne,Monterey Park
merchant_state,CA,CA


To sum up the data cleaning steps that will need to be repeated for the full training and test samples:
1. Renaming the columns.
1. Stripping the `$` and converting the transaction amounts to floating point values.
1. Creating a combined timestamp columns.
1. Creating categories and renaming the `use_chip` column.
1. Converting `is_fraud` into a boolean value.

### 2.2 The user records file

Information about each user is stored in a separate file, `sd254_users.csv`.

### 2.3 The card records file

Each user can have more than one card. Information about the cards is stored in `sd254_cards.csv`.

### 2.4 The transactions file

The file `credit-card-transactions-ibm_v2.csv` includes 20 million records of simulated transactions. Importing training and test samples will be covered below. For now, though, let's confirm that the records have the same structure as the sample above.

In [19]:
small_chunk_reader = data_utils.make_txdata_reader(chunksize = 100)
sample_df = next(small_chunk_reader)
print(sample_df.head(2).T)
del small_chunk_reader

                                  0                    1
User                              0                    0
Card                              0                    0
Year                           2002                 2002
Month                             9                    9
Day                               1                    1
Time                          06:21                06:42
Amount                      $134.09               $38.48
Use Chip          Swipe Transaction    Swipe Transaction
Merchant Name   3527213246127876953  -727612092139916043
Merchant City              La Verne        Monterey Park
Merchant State                   CA                   CA
Zip                         91750.0              91754.0
MCC                            5300                 5411
Errors?                         NaN                  NaN
Is Fraud?                        No                   No


In [20]:
def clean_tx_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Carries out the operations to clean the transactions data frame:
    - change column names
    - convert `amount` to float
    - make a timestamp column
    - make `tx_type` categorical feature
    - convert `is_fraud` to bool
    """
    # Updating the column names
    old_colnames = df.columns
    new_colnames = (
        old_colnames
        .str.lower()
        .str.replace(' ', '_', regex=False)
        .str.replace('?', '', regex=False)
    )
    df.columns = new_colnames
    # Converting transactions to float
    df.amount = df.amount.str.slice(1)\
        .astype('float')
    # Making a timestamp column
    df['timestamp'] = df.apply(row_to_timestamp, axis=1)
    # Creating the tx_type categorical feature
    df['tx_type'] = df.use_chip\
        .str.strip(" Transaction")\
        .str.lower()\
        .astype("category")
    # Converting the is_fraud to bool
    df.is_fraud = df.is_fraud == 'Yes'
    return df


In [21]:
clean_tx_df(sample_df).head(2).T

,0,1
user,0,0
card,0,0
year,2002,2002
month,9,9
day,1,1
time,06:21,06:42
amount,134.09,38.48
use_chip,Swipe Transaction,Swipe Transaction
merchant_name,3527213246127876953,-727612092139916043
merchant_city,La Verne,Monterey Park


## 3 Sampling Strategies

The main transactions data file includes 20 million records. This is too large a scale for the limited computing resources available for this project.